# An Experimental Machine Learning Study of High Achievers at the Open University using the Learning Analytics Data Set
Key findings:
- Variance of click times is the best predictor of high achievement controlling for skewness and kurtosis when it comes to model gain, suggesting that spreading one's study time out is the most important predictor of success. This was usually followed by certain types of engagement (e.g. with quizzes and forums) and one's previous education level.

## Introduction

The Open University is designed for those who wish to study a degree but whose cirumstances do not allow for the typical on-campus school 
leaver experience. Many students are significantly older than typical undergraduates, and many study online and part-time, receiving a degree in six to eight years.

In 2017 the University made public a large database. It contained information on student enrolment and demographics, but also kept track of all times a student interacted with the virtual learning environment (VLE).


## Decision Trees and XGBoost
A 'decision tree' in a machine learning context refers to a set of nodes which 'branch' based on a 'split', where the split aims to maximise the information gain from classifying the data within an existing node. Decision trees can be grouped in a 'tree ensemble', most simply in the Random Forest algorithm which takes a set of decision trees based on sampling with replacement and uses a vote system to make predictions.
'XGBoost' is a further modification which weights previously misclassified examples more heavily. It is highly robust and reliable, and performs well on the Learning Analytics Data Set in particular.


## The focus of the project
A lot of data on unequal outcomes focusses on average scores (including Nottingham’s own dashboards). Work with the OULA database schema itself also tends to focus on predicting 'performance' in general, or 'dropping out'. I wanted to focus on something new; the highest achievers and, specifically, whether having a previous degree makes a substantial difference to the probability of achieving highly. I have focussed on those who received a distinction in the module the first time they took it.

At the advice of the Open University itself, I analyse the February and October presentations separately as their content and presentation structure often differ from each other.



## Database Creation
First, I created the necessary tables in SQL. I mostly use the recommendations on the University website, although sometimes these do not make logical sense so I use my own judgement.



In [ ]:
--Example of table creation (studentInfo)
CREATE TABLE studentInfo (code_module VARCHAR(45),
	code_presentation VARCHAR(45),
	id_student INT,
	gender VARCHAR(3),
	imd_band VARCHAR(16),
	highest_education VARCHAR(45),
	age_band VARCHAR(16),
	number_of_prev_attempts INT,
	studied_credits INT,
	region VARCHAR(45),
	disability VARCHAR(3),
	final_result VARCHAR(45),
	PRIMARY KEY (id_student, code_module, code_presentation),
	FOREIGN KEY (code_module, code_presentation) 
	REFERENCES courses(code_module, code_presentation))

## Data Load
Next, I loaded the data into PostgreSQL using pgadmin4. This involved surprisingly little data cleaning beforehand. Some of the variable titles in the csv didn't match the OU website's description, so I had to amend these. There were also some extra columns present in the actual CSVs, so I had to alter and update the tables within SQL. 



## Data Transformation
As the data is presented over seven tables, I had to transform the learning engagement clicks data into a usable format for XGBoost. I used this loop to get the sum of clicks of each type of engagement. Not all of these types of engagement are present in every module. Later on, in python, I will automatically filter this inforamtion such that only resources available for the module in question are considered as features.


In [ ]:
--Loops through each type of learning resource and finds the sum of clicks for each student on that resource
DO $$

DECLARE
	activity RECORD;
	col_name VARCHAR(45);
BEGIN
	FOR activity in 
		SELECT DISTINCT activity_type
		FROM vle
	LOOP
		col_name := 'total_clicks_' || activity.activity_type;
	
		EXECUTE format('ALTER TABLE studentInfo
					   ADD COLUMN %I INT DEFAULT 0;', col_name);
		
		EXECUTE format('UPDATE studentInfo
					   SET %I = subquery.total_clicks
					   	FROM (SELECT id_student, code_module, code_presentation, SUM(sum_click)
						AS total_clicks
					   	FROM studentVle
					   WHERE activity_type = %L
					   GROUP BY id_student, code_module, code_presentation) AS subquery
					   WHERE studentInfo.id_student = subquery.id_student
              		   AND studentInfo.code_module = subquery.code_module
             		   AND studentInfo.code_presentation = subquery.code_presentation;',
					   col_name, activity.activity_type
					   );
		END LOOP;

END $$

## Joining modules
The Open University explains that modules are run bimonthly, but within a year are presented differently. Thus, I combined the modules between years but only WITHIN time of year.

In [ ]:
--Joins together modules where their presentation is the same
UPDATE studentInfo
SET pres_groups =
  CASE
    WHEN code_module = 'AAA' AND code_presentation = '2013J' THEN 0
    WHEN code_module = 'AAA' AND code_presentation = '2014J' THEN 0
    WHEN code_module = 'BBB' AND code_presentation = '2013B' THEN 1
	WHEN code_module = 'BBB' AND code_presentation = '2014B' THEN 1
	WHEN code_module = 'BBB' AND code_presentation = '2014J' THEN 2
    WHEN code_module = 'BBB' AND code_presentation = '2013J' THEN 2
    WHEN code_module = 'CCC' AND code_presentation = '2014B' THEN 3
    WHEN code_module = 'CCC' AND code_presentation = '2014J' THEN 4
    WHEN code_module = 'DDD' AND code_presentation = '2013B' THEN 5
	WHEN code_module = 'DDD' AND code_presentation = '2014B' THEN 5
    WHEN code_module = 'DDD' AND code_presentation = '2013J' THEN 6
    WHEN code_module = 'DDD' AND code_presentation = '2014J' THEN 6
    WHEN code_module = 'EEE' AND code_presentation = '2013J' THEN 7
	WHEN code_module = 'EEE' AND code_presentation = '2014J' THEN 7
    WHEN code_module = 'EEE' AND code_presentation = '2014B' THEN 8
    WHEN code_module = 'FFF' AND code_presentation = '2013B' THEN 9
    WHEN code_module = 'FFF' AND code_presentation = '2014B' THEN 9
	WHEN code_module = 'FFF' AND code_presentation = '2013J' THEN 10
    WHEN code_module = 'FFF' AND code_presentation = '2014J' THEN 10
    WHEN code_module = 'GGG' AND code_presentation = '2013J' THEN 11
	WHEN code_module = 'GGG' AND code_presentation = '2014J' THEN 11
    WHEN code_module = 'GGG' AND code_presentation = '2014B' THEN 12
    ELSE NULL
  END

## Static Engagement Variables

I have designed a number of features relating to engagement with the Virtual Learning Environment. To start with, I have merged two tables to create variables 'total clicks' of each assessment type. This is essentially the total number of times, for each student's performance in a particular module, the student clicks on e.g. quizzes. For example, student '111568' might have clicked on resource 5540 (quizzes) a total of 11 times during their attempt at module AAA. You will notice the data is essentially entirely anonymised, even the module names! 

In [ ]:
--Finds number of clicks by each type of resource
DO $$
DECLARE
	activity RECORD;
	col_name VARCHAR(45);
BEGIN
	FOR activity in 
		SELECT DISTINCT activity_type
		FROM vle
	LOOP
		col_name := 'total_clicks_' || activity.activity_type;
	
		EXECUTE format('ALTER TABLE studentInfo
					   ADD COLUMN %I INT DEFAULT 0;', col_name);
		
		EXECUTE format('UPDATE studentInfo
					   SET %I = subquery.total_clicks
					   	FROM (SELECT id_student, code_module, code_presentation, SUM(sum_click)
						AS total_clicks
					   	FROM studentVle
					   WHERE activity_type = %L
					   GROUP BY id_student, code_module, code_presentation) AS subquery
					   WHERE studentInfo.id_student = subquery.id_student
              		   AND studentInfo.code_module = subquery.code_module
             		   AND studentInfo.code_presentation = subquery.code_presentation;',
					   col_name, activity.activity_type
					   );
		END LOOP;

END $$

## Python Data Preparation

This concluded the bulk of the work in SQL. Following this, I imported various modules into python and connected the SQL database. 
I transformed income decile into a 1-10 scale, and deleted all attempts that were not the student's first - remember, my measure of 'high achievement' is getting a distinction first time.

A cursory EDA showed that we have around 32,000 participants, and each module has several hundred students. For most modules, a lot of types of resource are zero across the board. However, certain types, such as forums, are universal. The average number of total 'clicks' of any kind per module is 739. The 25th percentile is 260, and the 75th is 1770. This all suggests that the majority of students are engaging in some capacity, though the extent to which these are 'compulsory' clicks is unclear. It is far beyond the scope of the project to assess demographic variables one by one. However, it's worth stating that around 10% of students fall into the category of getting a distinction in the module first time.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import psycopg2 as pg2
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, precision_score, recall_score, f1_score
from xgboost import XGBClassifier, plot_importance

#Reads SQL table of Student Information
secret = '1234'
conn = pg2.connect(database='ou_la', user = 'postgres', password = secret)
cur = conn.cursor()
cur.execute('SELECT * FROM studentInfo')
print(cur.fetchmany(3))
df = pd.read_sql_query("SELECT * FROM studentInfo", conn)

#Exploratory Stage
print(df.head())
print(df.shape)
print(df.describe())

pres_modules = {
    'AAA-2013J': 0,
    'AAA-2014J': 0,
    'BBB-2013B': 1,
    'BBB-2013J': 2,
    'BBB-2014B': 1,
    'BBB-2014J': 2,
    'CCC-2014B': 3,
    'CCC-2014J': 4,
    'DDD-2013B': 5,
    'DDD-2013J': 6,
    'DDD-2014B': 5,
    'DDD-2014J': 6,
    'EEE-2013J': 7,
    'EEE-2014B': 8,
    'EEE-2014J': 7,
    'FFF-2013B': 9,
    'FFF-2013J': 10,
    'FFF-2014B': 9,
    'FFF-2014J': 10,
    'GGG-2013J': 11,
    'GGG-2014B': 12,
    'GGG-2014J': 11
}

imd_dic = {
        '0-10%': 1,
        '10-20%': 2,
        '20-30%': 3,
        '30-40%': 4,
        '40-50%': 5,
        '50-60%': 6,
        '60-70%': 7,
        '70-80%': 8,
        '80-90%': 9,
        '90-100%': 10,
    }

df['student_and_module'] = df['id_student'].astype(str) + df['code_module']
df['module_and_presentation'] = df['code_module'] + "-" + df['code_presentation']
df['pres_groups'] = df['module_and_presentation'].map(pres_modules)
df['dep_dec'] = df['imd_band']
df['dep_dec'] = df['dep_dec'].map(imd_dic)

#Only keep students' first attempt
df_earliest = df.drop_duplicates(subset = 'student_and_module', keep = 'first')
df_first = df_earliest[df_earliest['number_of_prev_attempts'] == 0]

df_first['distinction'] = np.where(df_first['final_result'] == 'Distinction', 1, 0)
print(df_first['distinction'].sum())
print((df_first['pres_groups'] == 2).sum())

#One Hot Code Categorical Variables
df_first_OHE = pd.get_dummies(df_first, columns = ['gender', 'highest_education', 'disability', 'age_band', 'region'], prefix = ['gender', 'highest_education', 'disability', 
                                                                                                                                 'age_band', 'region'])

## Dynamic Engagement Variables

The data frame also contains the date at which the student clicked on the resource, relative to when the module started. This allows for the number of clicks of each type to be analysed, as well as the time dynamics. I realised while brainstorming that there's a pretty good theoretical rationale for all statistical 'moments' when it comes to the day on which resources were clicked. This isn't based on a literature review of any kind (I don't consider this a hugely academic project, far more a personal one) but my thinking was as follows:
- The variance (known as the second moment) is of course important. I expect that a student who spreads their study across the year is likely to do better than one who does not.
- The skewness (the third moment) should be too. A student whose studying is all done at the start, or who leaves their studying until the last few days, is also likely not to perform so well. 
- The kurtosis (the fourth moment) looks at the thickness of tails specifically. Someone whose studying is only done at the tails might also be less likely to excel, in a way that won't be picked up by simple variance or skewness towards one end or the other.

Given that the skewness and kurtosis variables in scipy do not have options for weighting, I created variables in python with weights for the number of clicks for that day, using the mathematical formulas for said moments. I also included the coefficient of variation, as might further capture students whose variance is spread out but not clustered at the tails.

In [ ]:
## Weighted variance, skewness and kurtosis
def calculate_weighted_moments(group):
    days = group['click_date']
    clicks = group['sum_click']
    
    # Weighted mean
    weighted_mean = (days * clicks).sum() / clicks.sum()

    # Weighted variance
    weighted_variance = ((clicks * (days - weighted_mean)**2).sum() / clicks.sum())

    if weighted_mean == 0:
        coef_of_var = np.nan  # Set CV to NaN if weighted mean is zero
    else:
        coef_of_var = (weighted_variance ** 0.5) / weighted_mean

    # Check for zero or NaN variance
    if weighted_variance == 0 or pd.isna(weighted_variance):
        # Return NaN for skewness and kurtosis if variance is zero
        return pd.Series({
            'click_time_variance': weighted_variance,
            'click_time_skewness': np.nan,
            'click_time_kurtosis': np.nan,
            'click_time_cv': np.nan
        })

    weighted_skewness = ((clicks * (days - weighted_mean)**3).sum() / (clicks.sum() * (weighted_variance ** (3/2))))

    # Weighted kurtosis
    weighted_kurtosis = ((clicks * (days - weighted_mean)**4).sum() / (clicks.sum() * weighted_variance**2)) - 3

    return pd.Series({
        'click_time_variance': weighted_variance,
        'click_time_skewness': weighted_skewness,
        'click_time_kurtosis': weighted_kurtosis,
        'click_time_cv': coef_of_var
    })

## Integration with SQL Database

Following this, I added the variables to the student information able and merged the two datasets. Then, I took the relevant explanatory variables - demographics, clicks, and the moments - to create the X data frame.

In [ ]:
sVle = pd.read_sql_query("SELECT * FROM studentVle", conn)
moments = sVle.groupby(['id_student', 'code_module', 'code_presentation']).apply(calculate_weighted_moments)
print(moments.head())

print(moments)

moments = moments.reset_index()

df_with_moments = pd.merge(df_first_OHE, moments, on=['id_student', 'code_module', 'code_presentation'], how='left')

# Check the merged DataFrame
print(df_with_moments.head())

print(df_first_OHE.columns.tolist())

X = df_with_moments[['studied_credits', 'total_clicks_url', 'total_clicks_forumng', 'total_clicks_homepage', 'total_clicks_oucontent', 'total_clicks_subpage', 
                'total_clicks_resource', 'total_clicks_sharedsubpage', 'total_clicks_page', 'total_clicks_questionnaire', 'total_clicks_ouwiki', 'total_clicks_htmlactivity', 
                'total_clicks_ouelluminate', 'total_clicks_dataplus', 'total_clicks_externalquiz', 'total_clicks_repeatactivity', 'total_clicks_dualpane', 'total_clicks_quiz', 
                'total_clicks_glossary', 'total_clicks_oucollaborate', 'total_clicks_folder', 'pres_groups', 'dep_dec',
                'distinction', 'gender_F', 'gender_M', 'highest_education_A Level or Equivalent', 'highest_education_HE Qualification', 'highest_education_Lower Than A Level', 
                'highest_education_No Formal quals', 'highest_education_Post Graduate Qualification', 'disability_N', 'disability_Y', 'age_band_0-35', 'age_band_35-55', 
                'age_band_55<=', 'region_East Anglian Region', 'region_East Midlands Region', 'region_Ireland', 'region_London Region', 'region_North Region', 
                'region_North Western Region', 'region_Scotland', 'region_South East Region', 'region_South Region', 'region_South West Region', 'region_Wales', 
                'region_West Midlands Region', 'region_Yorkshire Region', 'click_time_variance', 'click_time_skewness', 'click_time_kurtosis', 'click_time_cv']]

X.columns=X.columns.str.replace(' ', '_')
X.columns=X.columns.str.replace('<=', '+')
X.columns=X.columns.str.replace(',', '')

## Model Creation

I decided to use XGBoost across each module (broken down into groupings by half of year, as discussed above). I then do the same for all modules pooled together. I weighted sampling strongly towards the minority class. This appeared to work proportionately within-models (i.e. 9:1 for not receiving distinction to receiving) but not for all modules pooled together. A more moderate weighting of 3:1 worked best here.

In [ ]:
#For each model

precisions = []
recalls = []
F1s = []
for i in range(1, 13):
    X_temp = X[X['pres_groups'] == i].copy()
    X_temp = X_temp.loc[:, (X_temp != 0).any(axis=0)]
    print(X.head())
    y = X_temp['distinction']
    X_temp = X_temp.drop(columns = ['distinction', 'pres_groups'])
    X_train, X_test, Y_train, y_test = train_test_split(X_temp, y, test_size=0.2, random_state=1)

    model = XGBClassifier(use_label_encoder = False, 
                    eval_metric = 'logloss', 
                    scale_pos_weight = 9, 
                    max_depth = 3,
                    subsample = 0.5)
    model.fit(X_train, Y_train)
    y_pred = model.predict(X_test)
    #brief attempt at changing decision threshold
    y_pred = (model.predict_proba(X_test)[:,1] >= 0.3).astype(int)
    print(classification_report(y_test, y_pred))
    F1s.append(f"F1 scores for module type {i} = {tuple(f1_score(y_test, y_pred, average=None))}")
    precisions.append(f"Precision scores for module type {i} = {tuple(precision_score(y_test, y_pred, average=None))}")
    recalls.append(f"Recall scores for module type {i} = {tuple(recall_score(y_test, y_pred, average=None))}")
    print(precisions)
    print(recalls)
    print(F1s)    
    plt.rcParams["figure.figsize"] = (18, 9)
    plot_importance(model, title=f"Feature Importance - module presentation {i}", max_num_features = 15, importance_type = "gain", values_format = "{v:.2f}")
    plt.show()

#Over all modules:
y = X['distinction']
X = X.drop(columns = ['distinction', 'pres_groups'])
X_train, X_test, Y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

model = XGBClassifier(use_label_encoder = False, 
                    eval_metric = 'logloss', 
                    scale_pos_weight = 2, 
                    max_depth = 3,
                    subsample = 0.5)
model.fit(X_train, Y_train)
y_pred = model.predict(X_test)
y_pred = (model.predict_proba(X_test)[:,1] >= 0.3).astype(int)
print(classification_report(y_test, y_pred))
precisions.append(f"Precision scores for all modules = {tuple(precision_score(y_test, y_pred, average=None))}")
recalls.append(f"Recall scores for all modules = {tuple(recall_score(y_test, y_pred, average=None))}")
F1s.append(f"F1 score over all modules = {tuple(f1_score(y_test, y_pred, average=None))}")
print(precisions)
print(recalls)
print(F1s)
plt.rcParams["figure.figsize"] = (18, 9)
plot_importance(model, title=f"Feature Importance - across all modules", importance_type = "gain", max_num_features = 15, xlabel = "Gain per variable", 
                ylabel = "Top 15 Features", values_format = "{v:.2f}")
plt.show()


# Results and Discussion

## Basic Results

## Feature Importance
When looking across all modules pooled together, it is immediately apparent that click_time_variance is the variable with by far the most gain - more than double any other variable. Given we are controlling for clustering in tails, this seems to suggest that those who spread out their studying gain the most. Next comes whether the student does not have A Levels. This is a fairly sad result - it seems to show that those student coming from lower educational backgrouds struggle to achieve. Of the next most imporant ten or so, the vast majority are different types of engagement with the online learning environments (with some exceptions like the odd regional dummy, which is probably beyond my scope to give insight into). Engagement with forums, the open university wiki, quizzes and the homepage itself seem to be most important - and crucially, more important than demographic factors such as gender, disability and income decile.



![Most important fifteen features across all modules combined](feature_importance.png)

## Predictive Power
The goal of this project is not so much to make successful predictions as to assess which demographic variables and factors contribute most to high achievement. However, it is worth discussing in basic terms anyway.
First, I try with just default parameters and with only demographic characteristics and the 'sum of clicks' for each resource. Here, the F1 score for not receiving a distinction is 0.96, and for receiving is 0.18. 

By lowering the decision threshold I can get relatively high recall scores for both (just under 0.8) but at the expense of precision for non-distinction cases. There was very little I could find that made the precision for non-distinction cases high, while small adjustments to the decision threshold improved the precision remarkably. The F-score for the non-distinction cases was hard to raise above around 0.4, which is still of course far higher than a random guessing algorithm but not as high as I would have hoped. 
So when I add my engagement features, and tune the hyperparameters, I can get to 0.90 and 0.41 F-scores respectively.
The recall is more even; I get close to 0.85 distinction and 0.63 for non-distinction, partly by lowering the decision threshold. However, the precision is extremely low consistently for the non-distinction class, and I did not manage to find an effective remedy to that. In other words, there are a very large number of false positives. While I get the majority of non-distinction cases, I also incorrectly assign distinction most of the time.



## Personal Reflections
This was my first real machine learning project that wasn't basic logistic regression or factor analysis from my quantitative social science degrees. If nothing else, it's absolutely essential that I did this project following my machine learning courses. I feel significantly more confident in my ability to create and apply machine learning models. It has reinforced my understanding of decision trees specifically, but also of general ML principles - bias and variance, feature engineering, parameter tuning, and of course the ML project life cycle, to name but a few.

## Discussion of Results

The interesting part, to me, is the feature importance. Variances scores highly in feature importance for essentially every module. Across all modules together, variance is the most important feature followed by previous education level and certain forms of engagement. Total clicks data is by some considerable margin more important than demographic features.

Clearly, the model struggles in some respects at making accurate predictions. One thing I would say is that the 'distinction first time' is effectively a binary variable drawn from a latent continuous distribution. Unlike binary classifications from binary options, there are a large number of edge cases. These will of course be hard to detect.
I did not try everything I could to get around this problem. It was outside the scope of the project to adjust more than a few hyperparameters, or to create synthetic minority samples.

Either way:

We have a significant improvement relative to a random guessing algorithm, and some very interesting results regarding feature importance. But clearly, if we are to try and make accurate predictions about students given their demographic data and engagement patterns, we need more than just total clicks and basic statistical properties of their engagement history. I strongly suspect that the engagement variables I created only scratch the surface of relevant patterns. Even so, it has been an interesting exercise. The sheer domination of click variance is interesting enough to make this project feel worth it even notwithstanding the skills I gained.